# Using Binance API feeding

Here goes the link, explanation and all that.
https://github.com/sammchardy/python-binance

In [2]:
# here goes all the imports just because
import json
from binance.client import Client
from externalized_functions import get_historical_klines, date_to_milliseconds, get_futures_historical_klines
import pandas as pd


print('feed me daddy')

feed me daddy


## Using a Client object to get historical data
It´s not necessary to create an API to get historical data from Binance, we can use a dummy client with
empty '' keys to get data.

In [3]:
# set some parameters
symbol = "BTCUSDT"  # trade-pair to extract
start = "9 Sep, 2019"  # yes
end = "1 Sep, 2020"  # uhu
interval = Client.KLINE_INTERVAL_1MINUTE  # granularity
fileout = "Binance_{}_{}_{}-{}.csv".format(symbol, interval, date_to_milliseconds(start), date_to_milliseconds(end))

# let's get it
lines = get_futures_historical_klines(symbol, interval, start, end)

# convert to dataframe so we can keep playing and save it to a csv file
columns = ['Open_time', 'Open', 'High', 'Low','Close', 'Volume', 'Close_time', 'Quote_asset_volume',
           'Number_of_trades', 'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'Ignore']
df = pd.DataFrame(lines, columns=columns)
df.to_csv(fileout, index=False)

## Plotting some data extracted

Let´s make some graphs just out of curiosity.

In [3]:
import finplot as fplt
import numpy as np
import pandas as pd

symbol = 'FUTURES BTC-USDT'

#df = pd.read_csv('Binance_BTCUSDT_1m_1596240000000-1598918400000.csv')
df = pd.read_csv('Binance_BTCUSDT_1m_1567987200000-1598918400000.csv')
df = df.rename(columns={'Open_time':'time', 'Open':'open', 'Close':'close', 'High':'high', 'Low':'low', 'Volume':'volume'})
#df = df.astype({'time':'datetime64[ns]'})
df
#df
#data
#print('here goes the graph')

,time,open,high,low,close,volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Ignore
0,1567987200000,10391.63,10391.63,10391.63,10391.63,0.000,1567987259999,0.000000e+00,0,0.000,0.000000e+00,0
1,1567987260000,10391.63,10391.63,10391.63,10391.63,0.000,1567987319999,0.000000e+00,0,0.000,0.000000e+00,0
2,1567987320000,10391.63,10391.63,10391.63,10391.63,0.000,1567987379999,0.000000e+00,0,0.000,0.000000e+00,0
3,1567987380000,10391.63,10391.63,10391.63,10391.63,0.000,1567987439999,0.000000e+00,0,0.000,0.000000e+00,0
4,1567987440000,10391.63,10391.63,10391.63,10391.63,0.000,1567987499999,0.000000e+00,0,0.000,0.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
515516,1598918160000,11666.84,11666.84,11658.77,11659.74,48.465,1598918219999,5.652471e+05,223,22.097,2.576819e+05,0
515517,1598918220000,11659.73,11660.04,11655.21,11655.22,51.387,1598918279999,5.990682e+05,245,11.315,1.318951e+05,0
515518,1598918280000,11655.22,11659.17,11654.00,11658.11,48.126,1598918339999,5.610001e+05,277,21.635,2.521977e+05,0
515519,1598918340000,11658.11,11660.53,11658.11,11658.11,78.462,1598918399999,9.147883e+05,245,21.574,2.515419e+05,0


In [4]:
# calculate indicators
window = 2
maxim = df['high'].rolling(window).max()
minim =  df['low'].rolling(window).min()
volatility = (maxim-minim)/maxim

#volatility = volatility.rolling(30).mean()
volatility = volatility.diff()

In [14]:
# create two plots
ax, ax2 = fplt.create_plot(symbol, rows=2)

# plot candle sticks
candles = df[['time','open','close','high','low']]
fplt.candlestick_ochl(candles, ax=ax)

# overlay volume on the top plot
volumes = df[['time','open','close','volume']]
fplt.volume_ocv(volumes, ax=ax.overlay())

# put an MA on the close price
fplt.plot(df['time'], df['close'].rolling(window).mean(), ax=ax, legend='ma-25')
fplt.plot(df['time'], maxim, ax=ax, legend='maxim')
fplt.plot(df['time'], minim, ax=ax, legend='minim')

# draw second plot
fplt.plot(df['time'], volatility, ax=ax2, color='#927', legend='volatility')
#fplt.set_y_range(0, 0.005, ax=ax2) # hard-code y-axis range limitation

# restore view (X-position and zoom) if we ever run this example again
fplt.autoviewrestore()

# we're done
fplt.show()

## Create an API on Binance
From your online Binance profile -> API Management, create an API and give it some label. An API key and secret key
will be generated, there you can config some restrictions like a hardcode on the IP address to access and 'read' or
'withdrawal' permissions.

Once the API is created I save it in a JSON and don't share it with anyone, it´s better if you save it because the
secret key will be visible for sometime and then forever hidden, to dump in a json you can use the following simple
code:

In [ ]:
# create a json file with my keys
api_key = 'my_beautiful_key'
secret_key = 'life_is_wonderful'
filename = 'binance-python-credentials.json'

json.dump(dict(api_key=api_key, secret_key=secret_key), open(filename, 'w'))
